In [21]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets, models
from torchvision.io import read_image
import torchvision.transforms as T
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn import preprocessing
from PIL import Image

In [2]:
class DogDataset(Dataset):
    def __init__(self, pd, folder, transform):  #초기화
        self.pd_table = pd
        self.img_dir = folder
        self.transform = transform

    def __getitem__(self, index):  #image load
        img_path = self.img_dir + self.pd_table.file_name[index] + ".jpg"
        img = Image.open(img_path)
        label = self.pd_table.encoded[index]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.pd_table)    

In [27]:
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 29 * 29, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 120)

    def forward(self, x):
        #print(x.size())
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
'''

#model = Net()
model = models.resnet18(pretrained = True)

num_classes = 120
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [33]:
image_size = 512

transform = T.Compose([
    T.Resize([image_size, image_size]),
    T.ToTensor(),
    T.Normalize(mean = (0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

In [35]:
pd_table = pd.read_csv("D:/code/dog-breed-identification/labels.csv", names = ['file_name', 'label'], header = 0)

le = preprocessing.LabelEncoder()
targets = le.fit_transform(pd_table['label'])
pd_table['encoded'] = targets
print(pd_table['encoded'].max())

train_dataset = DogDataset(pd = pd_table, folder = "D:/code/dog-breed-identification/train/", transform = transform)
train_size = int(len(train_dataset)*0.8)
validation_size = len(train_dataset) - train_size

train_data, validation_data = random_split(train_dataset, [train_size, validation_size])

train_loader = DataLoader(train_data, batch_size = 16, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size = 16, shuffle=True)

#print(train_data.__getitem__(0))

119


In [36]:
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

model.to(device)

#from torchsummary import summary
#summary(model, input_size=(3, image_size, image_size))

num_epoch = 50

In [ ]:
best_loss = 10

for epoch in range(num_epoch):   # 데이터셋을 수차례 반복합니다.
    
    model.train()
    train_loss = 0.0
    for x,y in train_loader:
        x = x.type(torch.FloatTensor)
        x = x.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)

        outputs = model(x)
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    train_loss = train_loss/len(train_loader)
    
    validation_loss = 0.0
    accuracy = 0.0
    total = 0.0
    with torch.no_grad():
        model.eval()
        for x, y in validation_loader:
            x = x.type(torch.FloatTensor)
            x = x.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            outputs = model(x)
            loss = criterion(outputs, y)
            
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            accuracy += (predicted == y).sum().item()
            
        validation_loss = validation_loss/len(validation_loader)
        accuracy = accuracy/total
        
    if validation_loss < best_loss:
        best_loss = validation_loss
        torch.save(model, 'D:/code/dog-breed-identification/my_model.pt')
        
    print('epoch:' + str(epoch+1) + ' train_loss:%.3f validation_loss:%.3f val_accuracy: %.3f best_loss:%.3f'%(train_loss,validation_loss, accuracy, best_loss))        
    
print('Finished Training')

epoch:1 train_loss:2.352 validation_loss:2.278 val_accuracy: 0.400 best_loss:2.278
epoch:2 train_loss:1.458 validation_loss:2.035 val_accuracy: 0.450 best_loss:2.035
epoch:3 train_loss:1.012 validation_loss:2.102 val_accuracy: 0.445 best_loss:2.035
epoch:4 train_loss:0.672 validation_loss:2.533 val_accuracy: 0.419 best_loss:2.035
epoch:5 train_loss:0.500 validation_loss:3.221 val_accuracy: 0.353 best_loss:2.035
epoch:6 train_loss:0.430 validation_loss:3.030 val_accuracy: 0.367 best_loss:2.035
epoch:7 train_loss:0.369 validation_loss:2.843 val_accuracy: 0.403 best_loss:2.035
epoch:8 train_loss:0.403 validation_loss:3.459 val_accuracy: 0.363 best_loss:2.035
epoch:9 train_loss:0.306 validation_loss:3.340 val_accuracy: 0.368 best_loss:2.035
epoch:10 train_loss:0.256 validation_loss:3.115 val_accuracy: 0.359 best_loss:2.035
epoch:11 train_loss:0.276 validation_loss:3.293 val_accuracy: 0.345 best_loss:2.035
epoch:12 train_loss:0.296 validation_loss:3.402 val_accuracy: 0.347 best_loss:2.035
e

In [17]:
print(predicted == y)

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False], device='cuda:0')
